In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.svm import LinearSVR

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import pickle
import json


In [2]:
## Bazı ilçelerde ev özellikleri değişebilmektedir. 
## Örneğin bir ilçede 6+1 odalı evler varken başka bir ilçede hiç 6+1 odalı ev olmayabilir.
## Bu nedenle her ilçede hangi özelliklerin bulunduğunun ayrıştırılması gerekir.

ILCE_LIST = ["esenyurt", "besiktas", "pendik", "kucukcekmece", "bahcelievler", "uskudar", "kadikoy", "sariyer", "bakirkoy", "sisli"]
roomCount = {}
buildingAge = {}
heatingType = {}


for ilce in ILCE_LIST:  
  houses_df = pd.read_csv("dataset/" + ilce + ".csv")
  roomCount[ilce] = houses_df['Oda Sayısı'].unique().tolist()
  buildingAge[ilce] = houses_df['Bina Yaşı'].unique().tolist()
  heatingType[ilce] = houses_df['Isıtma Tipi'].unique().tolist()

with open('roomCount.json', 'w') as fp:
    json.dump(roomCount, fp)

with open('buildingAge.json', 'w') as fp:
    json.dump(buildingAge, fp)

with open('heatingType.json', 'w') as fp:
    json.dump(heatingType, fp)

  


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/dataset/esenyurt.csv'

In [15]:
## Mobil uygulamadan gelen kullanıcı girişini model girdisine dönüştürmek için her ilçenin default giriş değeri oluşturulmuştur.
ILCE_LIST = ["esenyurt", "besiktas", "pendik", "kucukcekmece", "bahcelievler", "uskudar", "kadikoy", "sariyer", "bakirkoy", "sisli"]
DEFAULT_INPUTS = {}
for ilce in ILCE_LIST:  
  houses_df = pd.read_csv("dataset/" + ilce + ".csv")
  print(houses_df)
  houses_df = houses_df[houses_df['Kira'] < 50000]
  houses_df = houses_df[houses_df['Kira'] > 1000]
  houses_df = houses_df.reset_index(drop=True)
  
  oda_hashmap = {'6+1': 8, '2+0': 2,'2+2': 6, '1.5+1': 2, '2.5+1': 4, '2+1': 4,'3+1': 5,'5+2': 9,'Stüdyo': 1,'1+1': 2,'3+2': 7,'1 Oda': 1,'4+1': 6,'5+1': 7,'4+2': 8,'8+ Oda': 13,'7+2': 11,'6+2': 10, '3.5+1': 5}
  bina_hashmap = {'21 Ve Üzeri': 25, '5-10': 7, '11-15': 13, '16-20': 18, '4': 4, '0 (Yeni)': 0, '1': 1, '3': 3, '2': 2}
  balkon_hashmap = {'Yok': -1, 'Var': 1}
  esya_hashmap = {'Boş': -1, 'Eşyalı': 1}
  site_hashmap = {'Evet': 1, 'Hayır': -1}
  houses_df = houses_df.replace({'Oda Sayısı': oda_hashmap})
  houses_df = houses_df.replace({'Bina Yaşı': bina_hashmap})
  houses_df = houses_df.replace({'Balkon Durumu': balkon_hashmap})
  houses_df = houses_df.replace({'Eşya Durumu': esya_hashmap})
  houses_df = houses_df.replace({'Site İçerisinde': site_hashmap})
  houses_df = pd.get_dummies(houses_df)
  houses_df.pop('Kira')
  print(houses_df.columns)
  DEFAULT_INPUTS[ilce] = houses_df[:1]

pickle.dump(DEFAULT_INPUTS, open('DEFAULT_INPUTS', 'wb'))


      Kira  Alan Oda Sayısı  Banyo Sayısı Bina Yaşı          Isıtma Tipi  \
0     3900   128        3+1             1     16-20     Merkezi Doğalgaz   
1     6000    88        1+1             1  0 (Yeni)       Kombi Doğalgaz   
2    16000   100        2+1             1  0 (Yeni)       Kombi Doğalgaz   
3     2300    90        2+1             1      5-10       Kombi Doğalgaz   
4     2450   105        2+1             1         4       Kombi Doğalgaz   
..     ...   ...        ...           ...       ...                  ...   
359   1200    80        2+1             1  0 (Yeni)       Kombi Doğalgaz   
360   4750   250        4+1             2      5-10       Kombi Doğalgaz   
361   2700   190        2+1             2      5-10       Kombi Doğalgaz   
362   5000    70        1+1             1  0 (Yeni)  Merkezi (Pay Ölçer)   
363   4950   105        2+1             1      5-10  Merkezi (Pay Ölçer)   

    Balkon Durumu Eşya Durumu Site İçerisinde  
0             Yok         Boş          

In [12]:
## Bu kısımda veri setleri ile her bir ilçe için 4 adet model eğitilmiştir.

import pickle 
from tabulate import tabulate
final_table = []
ILCE_LIST = ["esenyurt", "besiktas", "pendik", "kucukcekmece", "bahcelievler", "uskudar", "kadikoy", "sariyer", "bakirkoy", "sisli"]

for ilce in ILCE_LIST:
  houses_df = pd.read_csv("dataset/" + ilce + ".csv")
  houses_df.head()
  houses_df = houses_df[houses_df['Kira'] < 50000]
  houses_df = houses_df[houses_df['Kira'] > 1000]
  houses_df = houses_df.reset_index(drop=True)
  oda_hashmap = {'6+1': 8, '2+0': 2,'2+2': 6, '1.5+1': 2, '2.5+1': 4, '2+1': 4,'3+1': 5,'5+2': 9,'Stüdyo': 1,'1+1': 2,'3+2': 7,'1 Oda': 1,'4+1': 6,'5+1': 7,'4+2': 8,'8+ Oda': 13,'7+2': 11,'6+2': 10, '3.5+1': 5}
  bina_hashmap = {'21 Ve Üzeri': 25, '5-10': 7, '11-15': 13, '16-20': 18, '4': 4, '0 (Yeni)': 0, '1': 1, '3': 3, '2': 2}
  balkon_hashmap = {'Yok': -1, 'Var': 1}
  esya_hashmap = {'Boş': -1, 'Eşyalı': 1}
  site_hashmap = {'Evet': 1, 'Hayır': -1}
  houses_df = houses_df.replace({'Oda Sayısı': oda_hashmap})
  houses_df = houses_df.replace({'Bina Yaşı': bina_hashmap})
  houses_df = houses_df.replace({'Balkon Durumu': balkon_hashmap})
  houses_df = houses_df.replace({'Eşya Durumu': esya_hashmap})
  houses_df = houses_df.replace({'Site İçerisinde': site_hashmap})
  houses_df.head()
  houses_df = pd.get_dummies(houses_df)
  houses_df_train, houses_df_test = train_test_split(houses_df, 
                                     train_size = 0.9, 
                                     test_size = 0.1, 
                                     random_state = 10)
  X_train = houses_df_train ## Kira harici kategoriler
  y_train = X_train.pop('Kira') ## Kira kategorisi
  X_test = houses_df_test
  y_test = X_test.pop('Kira')
  models = [LinearRegression(),
              RandomForestRegressor(n_estimators=100, max_features='sqrt'),
              KNeighborsRegressor(n_neighbors=1),
              SVR(kernel='linear'),
              ]
  trained_models=[]
  scores = [ilce]
  for model in models:
    reg_model = model.fit(X_train, y_train)
    trained_models.append(reg_model)
    y_pred = reg_model.predict(X_train)
    final_score = r2_score(y_train, y_pred)
    scores.append("{:.3f}".format(final_score))

  pickle.dump(trained_models, open("models/" + ilce, 'wb'))
  final_table.append(scores)

print(tabulate(final_table, headers = ["İlçe", "Linear Regression", "Random Forest Regressor", "KNeighborsRegressor", "SVR"]))

İlçe            Linear Regression    Random Forest Regressor    KNeighborsRegressor    SVR
------------  -------------------  -------------------------  ---------------------  -----
esenyurt                    0.169                      0.868                  0.962  0.084
besiktas                    0.547                      0.914                  0.937  0.295
pendik                      0.813                      0.848                  0.995  0.3
kucukcekmece                0.643                      0.947                  0.997  0.296
bahcelievler                0.508                      0.904                  0.964  0.307
uskudar                     0.76                       0.932                  0.954  0.276
kadikoy                     0.735                      0.932                  0.997  0.605
sariyer                     0.597                      0.941                  0.985  0.481
bakirkoy                    0.717                      0.896                  0.822  0.502
s